In [1]:
import numpy as np
from scipy.signal import convolve
from IPython.display import Audio
import matplotlib.pyplot as plt
import plotly.graph_objs as go
%matplotlib inline 
import h5py
from pydub import AudioSegment
import librosa
import soundfile as sf
import pyloudnorm as pyln

## Downmix the 5.1 audio to stereo

### Split to mono

In [2]:
path = "/Users/vio/Desktop/Surround.wav"
surround_sound = AudioSegment.from_file(path, format="wav")

### Assign channels direction

In [3]:
channels = surround_sound.split_to_mono()
front_left = channels[0]
front_right = channels[1]
center = channels[2]
lfe = channels[3]
surround_left = channels[4]
surround_right = channels[5]

In [4]:
type(front_left)

pydub.audio_segment.AudioSegment

### Playback each channel to check direction

In [5]:
# Firstly, convert audioSegment to a NumPy array
def segment_to_nparray(audio_segment):
    # Pydub audio segments are in milliseconds -> have to convert them to samples:
    # AudioSegment's .get_array_of_samples() returns the audio data as an array of samples.
    samples = np.array(audio_segment.get_array_of_samples())
    
    if audio_segment.sample_width == 2:
        samples = samples.astype(np.int24)
    return samples


In [ ]:
# Play the front left channel
front_left_np = segment_to_nparray(channels[0])
Audio(front_left_np, rate=channels[0].frame_rate)

In [ ]:
# Play the front right channel
front_right_np = segment_to_nparray(channels[1])
Audio(front_right_np, rate=channels[1].frame_rate)

In [ ]:
# Center
# Notice: 2:05 has no music element
center_np = segment_to_nparray(channels[2])
Audio(center_np, rate=channels[2].frame_rate)

In [ ]:
# Play the lfe
# Found that it's mostly silent except for some sub effect like 3:45
lfe_np = segment_to_nparray(channels[3])
Audio(lfe_np, rate=channels[3].frame_rate)

In [ ]:
# Play surround_left
# Notice: Less drums (e.g. at 2:25)
surround_left_np = segment_to_nparray(channels[4])
Audio(surround_left_np, rate=channels[4].frame_rate)

In [ ]:
# Play surround_right
# Notice: Less drums
surround_right_np = segment_to_nparray(channels[5])
Audio(surround_right_np, rate=channels[5].frame_rate)

### Create stereo mix

In [12]:
center_left = center.pan(-1)  # Full left
center_right = center.pan(1)  # Full right

# Overlay the left channels to create the left tracks; and so does the right.
left = front_left.overlay(center_left).overlay(surround_left)
right = front_right.overlay(center_right).overlay(surround_right)

# Ensure the tracks are mono by converting them explicitly
left = left.set_channels(1)
right = right.set_channels(1)

# create stereo mix
stereo_mix = AudioSegment.from_mono_audiosegments(left, right)

# Output and save
stereo_mix.export("stereo_downmix.wav", format="wav")

<_io.BufferedRandom name='stereo_downmix.wav'>

### Read HRIR

In [13]:
with h5py.File('/Users/vio/Downloads/D2_HRIR_SOFA/D2_48K_24bit_256tap_FIR_SOFA.sofa', 'r') as f:
    hrir_data = f['Data.IR'][:]  # Get HRIR/BRIR data
    sampling_rate = f['Data.SamplingRate'][:]
    listener_position = f['ListenerPosition'][:]
    source_positions = f['SourcePosition'][:]
    print(f"HRIR Data Shape: {hrir_data.shape}")
    print(f"Sampling Rate: {sampling_rate[0]} Hz")
    print(f"Listener Position: {listener_position}")
    print(f"Source Positions: {source_positions}")
    print("HRIR Data Shape (number of measurements, stereo, n_samples):", hrir_data.shape)

HRIR Data Shape: (8802, 2, 256)
Sampling Rate: 48000.0 Hz
Listener Position: [[0. 0. 0.]]
Source Positions: [[  0.  -90.    1.2]
 [  0.  -81.    1.2]
 [  0.  -75.    1.2]
 ...
 [359.   60.    1.2]
 [359.   64.8   1.2]
 [359.   75.    1.2]]
HRIR Data Shape (number of measurements, stereo, n_samples): (8802, 2, 256)


### Get the standard 5.1 speaker placement

In [14]:
# Front Left and Front Right Speakers
# Azimuth: +30 degrees and -30 degrees relative to the center front (0 degrees)

#Target [  +30.  0.    1.2] and [  -30.  0.    1.2]
target_positions = [[30, 0, 1.2], [330, 0, 1.2]]

# Iterate over target positions and find the indices and HRIR data
for pos in target_positions:
    # Find indices where source position matches the target position
    indices = np.where((source_positions[:, 0] == pos[0]) &
                        (source_positions[:, 1] == pos[1]) &
                        (source_positions[:, 2] == pos[2]))[0]

    if len(indices) > 0:
        # Get the HRIR data for the found indices
        for index in indices:
            hrir_for_position = hrir_data[index]
            print(f"HRIR for position {pos}: index {index}, shape {hrir_for_position.shape}")
    else:
        print(f"No HRIR found for position {pos}")

HRIR for position [30, 0, 1.2]: index 760, shape (2, 256)
HRIR for position [330, 0, 1.2]: index 8064, shape (2, 256)


In [15]:
# Center
# Target [  0.  0.    1.2]
center_position = [0, 0, 1.2]
indices = np.where((source_positions[:, 0] == center_position[0]) &
                       (source_positions[:, 1] == center_position[1]) &
                       (source_positions[:, 2] == center_position[2]))[0]

if len(indices) > 0:
    # Get the HRIR data for the found indices
    for index in indices:
        hrir_for_center = hrir_data[index]
        print(f"HRIR for center position: index {index}, shape {hrir_for_center.shape}")
else:
    print("No HRIR found for center position")

HRIR for center position: index 12, shape (2, 256)


In [16]:
# LFE
# Target [  15.  -15.    1.2]
lfe_position = [15, -15, 1.2]
indices = np.where((source_positions[:, 0] == lfe_position[0]) &
                       (source_positions[:, 1] == lfe_position[1]) &
                       (source_positions[:, 2] == lfe_position[2]))[0]

if len(indices) > 0:
    # Get the HRIR data for the found indices
    for index in indices:
        hrir_for_lfe = hrir_data[index]
        print(f"HRIR for lfe position: index {index}, shape {hrir_for_lfe.shape}")
else:
    print("No HRIR found for center position")

HRIR for lfe position: index 369, shape (2, 256)


In [17]:
# SL and SR
# Target [  +120.  15.    1.2] and [  -120.  15.    1.2]
rear_positions = [[120, 15, 1.2], [240, 15, 1.2]]

for pos in rear_positions:
    # Find indices where source position matches the target position
    indices = np.where((source_positions[:, 0] == pos[0]) &
                         (source_positions[:, 1] == pos[1]) &
                         (source_positions[:, 2] == pos[2]))[0]

    if len(indices) > 0:
         # Get the HRIR data for the found indices
        for index in indices:
            hrir_for_position = hrir_data[index]
            print(f"HRIR for position {pos}: index {index}, shape {hrir_for_position.shape}")
    else:
        print(f"No HRIR found for position {pos}")

HRIR for position [120, 15, 1.2]: index 2950, shape (2, 256)
HRIR for position [240, 15, 1.2]: index 5876, shape (2, 256)


### Plot the positions

In [18]:
positions = {
    (30, 0, 1.2): 'blue',    # Front right
    (330, 0, 1.2): 'blue',   # Front left (330 degrees is equivalent to -30 degrees)
    (0, 0, 1.2): 'blue',     # Center front
    (15, -15, 1.2): 'blue',  # Slightly front-right and below
    (120, 15, 1.2): 'blue',  # Surround left
    (240, 15, 1.2): 'blue',  # Surround right
    (0, 0, 0): 'red'         # Origin
}

# Convert each position to Cartesian coordinates for plotting
x = []
y = []
z = []
colors = []

for (azimuth, elevation, distance), color in positions.items():
    # Convert spherical to Cartesian coordinates
    azimuth_rad = np.radians(azimuth)
    elevation_rad = np.radians(elevation)
    x_coord = distance * np.cos(elevation_rad) * np.cos(azimuth_rad)
    y_coord = distance * np.cos(elevation_rad) * np.sin(azimuth_rad)
    z_coord = distance * np.sin(elevation_rad)

    # Append coordinates and color
    x.append(x_coord)
    y.append(y_coord)
    z.append(z_coord)
    colors.append(color)

# Create a 3D scatter plot with Plotly
scatter_plot = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=5,
        color=colors,  # Use the color array here
        opacity=0.8
    )
)

# Layout configuration
layout = go.Layout(
    title='Interactive 3D Plot of Specified Source Positions',
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-1.5, 1.5]),
        yaxis=dict(title='Y (meters)', range=[-1.5, 1.5]),
        zaxis=dict(title='Z (meters)', range=[-1.5, 1.5]),
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    )
)

# Create a figure and plot it
fig = go.Figure(data=[scatter_plot], layout=layout)
fig.show(renderer="browser")

### Load HRIR and Convert to 44.1kHz (to match my audio)

In [19]:
target_indices = [8064, 760, 12, 369, 5876, 2950]
with h5py.File('/Users/vio/Downloads/D2_HRIR_SOFA/D2_48K_24bit_256tap_FIR_SOFA.sofa', 'r') as f:
    hrir_data = f['Data.IR'][:]  # Get HRIR data
    original_sample_rate = f['Data.SamplingRate'][0]  # Get the original sampling rate

    # Prepare a dictionary to hold the resampled HRIRs
    resampled_hrir_data = {}

    # Process each target index
    for index in target_indices:
        # Load HRIRs for the specified index (ensure you understand the data shape and channel layout)
        hrir_left = hrir_data[index, 0, :]  # Assuming the first channel is left
        hrir_right = hrir_data[index, 1, :]  # Assuming the second channel is right

        # Resample HRIRs from original_sample_rate (48 kHz) to 44.1 kHz
        hrir_left_resampled = librosa.resample(hrir_left, orig_sr=original_sample_rate, target_sr=44100)
        hrir_right_resampled = librosa.resample(hrir_right, orig_sr=original_sample_rate, target_sr=44100)

        # Store resampled HRIRs
        resampled_hrir_data[index] = {
            'left': hrir_left_resampled,
            'right': hrir_right_resampled
        }

# Output the shapes of the resampled HRIRs to verify
for index, hrirs in resampled_hrir_data.items():
    print(f"Index {index} - Left HRIR shape: {hrirs['left'].shape}, Right HRIR shape: {hrirs['right'].shape}")

Index 8064 - Left HRIR shape: (236,), Right HRIR shape: (236,)
Index 760 - Left HRIR shape: (236,), Right HRIR shape: (236,)
Index 12 - Left HRIR shape: (236,), Right HRIR shape: (236,)
Index 369 - Left HRIR shape: (236,), Right HRIR shape: (236,)
Index 5876 - Left HRIR shape: (236,), Right HRIR shape: (236,)
Index 2950 - Left HRIR shape: (236,), Right HRIR shape: (236,)


### Convolution

In [20]:
def apply_hrir(audio_np, hrir_left, hrir_right):
    """
    Convolves stereo HRIR with a mono audio signal, returning stereo output.
    """
    left = np.convolve(audio_np, hrir_left, mode='full')[:len(audio_np)]
    right = np.convolve(audio_np, hrir_right, mode='full')[:len(audio_np)]
    return left, right

In [21]:
front_left_stereo = apply_hrir(front_left_np, resampled_hrir_data[8064]['left'], resampled_hrir_data[8064]['right'])
front_right_stereo = apply_hrir(front_right_np, resampled_hrir_data[760]['left'], resampled_hrir_data[760]['right'])
center_stereo = apply_hrir(center_np, resampled_hrir_data[12]['left'], resampled_hrir_data[12]['right'])
lfe_stereo = apply_hrir(lfe_np, resampled_hrir_data[369]['left'], resampled_hrir_data[369]['right']) 
surround_left_stereo = apply_hrir(surround_left_np, resampled_hrir_data[5876]['left'], resampled_hrir_data[5876]['right'])
surround_right_stereo = apply_hrir(surround_right_np, resampled_hrir_data[2950]['left'], resampled_hrir_data[2950]['right'])

In [22]:
final_left = front_left_stereo[0] + front_right_stereo[0] + center_stereo[0] + lfe_stereo[0] + surround_left_stereo[0] + surround_right_stereo[0]
final_right = front_left_stereo[1] + front_right_stereo[1] + center_stereo[1] + lfe_stereo[1] + surround_left_stereo[1] + surround_right_stereo[1]

In [23]:
def normalize_stereo(final_left, final_right):

    peak_amplitude = np.max(np.abs([final_left, final_right]))
    normalization_factor = 1.0 / peak_amplitude

    # Normalize both channels
    normalized_left = final_left * normalization_factor
    normalized_right = final_right * normalization_factor

    return normalized_left, normalized_right

In [24]:
normalized_left, normalized_right = normalize_stereo(final_left, final_right)

In [25]:
final_vss = np.vstack((normalized_left, normalized_right)).T
sf.write('final_vss_mix.wav', final_vss, 44100, 'PCM_24')

### The result is nice, but I want to make sure the LUFS stays consistent 
#### Actually forget about this part cuz I found that after loudness adjustment, the VSS auditory experience doesn't feel right.